In [1]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alloc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\alloc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [12]:
print('**importing pakages**')
import pandas as pd
import numpy as np
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import re
from string import digits

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn import metrics
from sklearn.naive_bayes import BernoulliNB
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import KFold, cross_val_score
from sklearn import linear_model
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC 


print('**Loading data**')
df = pd.read_csv('./reddit_train.csv')

print('**Preprocessing data**')
df['clean_comments'] = df['comments'].replace(r'http\S', '', regex=True).replace(r'www\S', '', regex=True)
df['clean_comments'] = df['clean_comments'].str.replace("[^0-9a-zA-Z#+_]", " ")
df['clean_comments'] = df['clean_comments'].str.replace("&(\w);", " ")
#df['clean_comments'] = df['clean_comments'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))
tokenized_comments = df['clean_comments'].apply(lambda x: x.split())

stemmer = PorterStemmer()
tokenized_comments = tokenized_comments.apply(lambda x: [stemmer.stem(word)for word in x])

lem = WordNetLemmatizer()
tokenized_comments = tokenized_comments.apply(lambda x: [lem.lemmatize(word) for word in x])
for i in range(len(tokenized_comments)):
    tokenized_comments[i] = ' '.join(tokenized_comments[i])

df['clean_comments'] = tokenized_comments


# count_vect = CountVectorizer( lowercase=True, preprocessor=None, tokenizer=None,
#                              stop_words='english',  ngram_range=(1,2),
#                              analyzer='word', max_df=0.5, min_df=3, max_features=20000, vocabulary=None,
#                              binary=True)
# count = count_vect.fit_transform(df['clean_comments'])
# train_data = count
tfidf_vectorizer = TfidfVectorizer(max_df=0.99, min_df=2, smooth_idf = False, norm='l2', ngram_range=(1,1), stop_words='english')

tfidf = tfidf_vectorizer.fit_transform(df['clean_comments'])
train_data = tfidf


LE = LabelEncoder()
df['label'] = LE.fit_transform(df['subreddits'])
train_label = df['label'].to_numpy()

print('**Spliting traing and validation data**')

X_train, X_valid, y_train, y_valid = train_test_split(train_data, train_label, test_size = 0.2, random_state = 42)



**importing pakages**
**Loading data**
**Preprocessing data**
**Spliting traing and validation data**


## OnevsRest

In [ ]:
print('**Fitting model**')
vs =OneVsRestClassifier(clf, n_jobs=-1,verbose=True)
vs.fit(X_train, y_train)
y_predict = vs.predict(X_valid)

accuracy = metrics.accuracy_score(y_valid, y_predict)
print('Accuracy: {:.3f}'.format(accuracy))

## SVM

In [33]:
print('**Fitting model**')
clf = LinearSVC(penalty='l2', loss='hinge', dual=True, tol=0.01, C=5000,
                multi_class='ovr', fit_intercept=False, intercept_scaling=10,
                class_weight=None, verbose=10, random_state=42, max_iter=100)
clf.fit(X_train, y_train)
y_predict = clf.predict(X_valid)

accuracy = metrics.accuracy_score(y_valid, y_predict)
print('Accuracy: {:.3f}'.format(accuracy))

**Fitting model**
[LibLinear]

C:\Users\alloc\Anaconda3\envs\comp551a1\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Accuracy: 0.065


In [ ]:
from sklearn.pipeline import Pipeline
clf_pipeline = Pipeline([('clf', LinearSVC(verbose=True))])
parameters_clf = {
                  'clf__tol': (1e-4,1e-5,1e-3),
                  'clf__C':(0.2,0.1,0.05),
                  'clf__dual':(True,False),
                   'clf__intercept_scaling':(2,5,10,20),
    
                      
 }
gs_clf_svm = GridSearchCV(clf_pipeline, parameters_clf, n_jobs=-1,verbose=10)
gs_clf_svm = gs_clf_svm.fit(X_train, y_train)
print(gs_clf_svm.best_score_)
print(gs_clf_svm.best_params_)

## DecisionTree

In [ ]:
print('**Fitting model**')

clf_dtc = DecisionTreeClassifier(criterion='gini', splitter='best',
                                 max_depth=None, min_samples_split=10, min_samples_leaf=15,
                                 min_weight_fraction_leaf=0.0, max_features=None, 
                                 random_state=None, max_leaf_nodes=None,
                                 min_impurity_decrease=0.0, min_impurity_split=None, class_weight=None, presort=False)
clf_dtc= clf_dtc.fit(X_train, y_train)
y_predict = clf_dtc.predict(X_valid)

accuracy = metrics.accuracy_score(y_valid, y_predict)
print('Accuracy: {:.4f}'.format(accuracy))

## SGD test

In [32]:
print('**Fitting model**')
from sklearn.linear_model import SGDClassifier
clf_sdg = SGDClassifier(loss='modified_huber', penalty='l2', alpha=0.001,
                        l1_ratio=0.15, fit_intercept=True, max_iter=100, tol=0.00005, shuffle=True, verbose=0,
                        epsilon=1, n_jobs=-1, random_state=True, learning_rate='optimal', eta0=1.0, power_t=0.5,
                        early_stopping=False,
                        validation_fraction=0.9, n_iter_no_change=70, class_weight=None, warm_start=True, average=False)
clf_sdg= clf_sdg.fit(X_train, y_train)
y_predict = clf_sdg.predict(X_valid)

accuracy = metrics.accuracy_score(y_valid, y_predict)
print('Accuracy: {:.4f}'.format(accuracy))

**Fitting model**


C:\Users\alloc\Anaconda3\envs\comp551a1\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


Accuracy: 0.1224


In [ ]:
from sklearn.pipeline import Pipeline
clf_pipeline = Pipeline([('clf', SGDClassifier( eta0=1.0))])
parameters_clf = {
                  'clf__alpha': (1e-4,7e-5,4e-4),
                  'clf__tol':(0.0001,0.00001,0.000001),
                 'clf__learning_rate':('optimal','adaptive','invscaling'),
                    'clf__average':(True,False),
    
                      
 }
gs_clf_svm = GridSearchCV(clf_pipeline, parameters_clf, n_jobs=-1,verbose=10)
gs_clf_svm = gs_clf_svm.fit(X_train, y_train)
print(gs_clf_svm.best_score_)
print(gs_clf_svm.best_params_)

## NB

In [27]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
clf = MultinomialNB(alpha=0.27, fit_prior=False, class_prior=None)
clf = clf.fit(X_train, y_train)
predicted = clf.predict(X_valid)
np.mean(predicted == y_valid)

0.5734285714285714

In [ ]:
from sklearn.pipeline import Pipeline
clf_pipeline = Pipeline([('clf', MultinomialNB())])
parameters_clf = {
                  'clf__alpha': (0.7,0.35,0.45,0.4),
                  'clf__fit_prior':(True,False),
                  ## 'clf__class_prior':(True,False),
    
                      
 }
gs_clf_svm = GridSearchCV(clf_pipeline, parameters_clf, n_jobs=-1,verbose=10)
gs_clf_svm = gs_clf_svm.fit(X_train, y_train)
print(gs_clf_svm.best_score_)
print(gs_clf_svm.best_params_)

## Best MAXDF

In [ ]:
maxDF = 0.01
best_maxDF=0.0
accuracy = 0.0
while maxDF <0.9:
    ##setting up tfidf with maxDF
    tfidf_vectorizer = TfidfVectorizer(max_df=maxDF, min_df=2, smooth_idf = True, norm='l2', ngram_range=(1, 5), stop_words='english')
    # TF-IDF feature matrix
    tfidf = tfidf_vectorizer.fit_transform(df['clean_comments'])
    train_data = tfidf
    LE = LabelEncoder()
    df['label'] = LE.fit_transform(df['subreddits'])
    train_label = df['label'].to_numpy()
    X_train, X_valid, y_train, y_valid = train_test_split(train_data, train_label, test_size = 0.2, random_state = 42)
    
   
    clf_sdg = SGDClassifier(loss='modified_huber', penalty='l2', alpha=0.0001,
                        l1_ratio=0.15, fit_intercept=True, max_iter=1000, tol=0.0001, shuffle=True, verbose=0,
                        epsilon=5, n_jobs=-1, random_state=None, learning_rate='optimal', eta0=1.0, power_t=0.5,
                        early_stopping=False,
                        validation_fraction=0.8, n_iter_no_change=50, class_weight=None, warm_start=False, average=False)
    clf_sdg= clf_sdg.fit(X_train, y_train)
    y_predict = clf_sdg.predict(X_valid)
    
    if (accuracy < metrics.accuracy_score(y_valid, y_predict)):
        accuracy = metrics.accuracy_score(y_valid, y_predict)
        best_maxDF=maxDF
    
    maxDF = maxDF + 0.1

In [ ]:
print(accuracy)
print(best_maxDF)

In [ ]:
best_maxDF

## Word2vec

In [14]:
import tqdm
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from sklearn import utils
import gensim
from gensim.models import doc2vec
from gensim.models.doc2vec import TaggedDocument
import re

def label_sentences(corpus, label_type):
    """
    Gensim's Doc2Vec implementation requires each document/paragraph to have a label associated with it.
    We do this by using the TaggedDocument method. The format will be "TRAIN_i" or "TEST_i" where "i" is
    a dummy index of the post.
    """
    labeled = []
    for i, v in enumerate(corpus):
        label = label_type + '_' + str(i)
        labeled.append(doc2vec.TaggedDocument(v, [label]))
    return labeled

X_train, X_valid, y_train, y_valid = train_test_split(df['clean_comments'], df['label'], random_state=0, test_size=0.3)
X_train = label_sentences(X_train, 'Train')
X_valid = label_sentences(X_valid, 'Test')
all_data = X_train + X_valid

In [15]:
model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, min_count=1, alpha=0.065, min_alpha=0.065)
model_dbow.build_vocab([x for x in tqdm(all_data)])

for epoch in range(30):
    model_dbow.train(utils.shuffle([x for x in tqdm(all_data)]), total_examples=len(all_data), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

100%|███████████████████████████████████████████████████████████████████████| 70000/70000 [00:00<00:00, 4377926.76it/s]


In [16]:
def get_vectors(model, corpus_size, vectors_size, vectors_type):
    """
    Get vectors from trained doc2vec model
    :param doc2vec_model: Trained Doc2Vec model
    :param corpus_size: Size of the data
    :param vectors_size: Size of the embedding vectors
    :param vectors_type: Training or Testing vectors
    :return: list of vectors
    """
    vectors = np.zeros((corpus_size, vectors_size))
    for i in range(0, corpus_size):
        prefix = vectors_type + '_' + str(i)
        vectors[i] = model.docvecs[prefix]
    return vectors
    
X_train = get_vectors(model_dbow, len(X_train), 300, 'Train')
X_valid = get_vectors(model_dbow, len(X_test), 300, 'Test')

In [ ]:
logreg = LogisticRegression(n_jobs=-1, C=1e5)
logreg.fit(train_vectors_dbow, y_train)
logreg = logreg.fit(train_vectors_dbow, y_train)
y_pred = logreg.predict(test_vectors_dbow)
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

## MLP

In [13]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(hidden_layer_sizes=(95,), activation='relu', solver='adam',
                    alpha=0.00005, batch_size=200, learning_rate='invscaling',
                    learning_rate_init=0.0009, power_t=0.9, max_iter=4, shuffle=True,
                    random_state=False, tol=0.1, verbose=True, warm_start=False,
                    momentum=0.9, nesterovs_momentum=True, early_stopping=False,
                    validation_fraction=0.8, beta_1=0.9, beta_2=0.99, epsilon=1e-8,
                    n_iter_no_change=1)
clf = clf.fit(X_train, y_train)
predicted = clf.predict(X_valid)
np.mean(predicted == y_valid)

Iteration 1, loss = 2.62307217
Iteration 2, loss = 1.70338398
Iteration 3, loss = 1.30452745
Iteration 4, loss = 1.09678012


C:\Users\alloc\Anaconda3\envs\comp551a1\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (4) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


0.586

In [ ]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', solver='adam',
                    alpha=0.0001, batch_size='auto', learning_rate='adaptive',
                    learning_rate_init=0.00, power_t=0.5, max_iter=8, shuffle=True,
                    random_state=None, tol=0.0001, verbose=True, warm_start=False,
                    momentum=0.9, nesterovs_momentum=True, early_stopping=False,
                    validation_fraction=0.3, beta_1=0.8, beta_2=0.99, epsilon=1e-08,
                    n_iter_no_change=10)
from sklearn.pipeline import Pipeline
clf_pipeline = Pipeline([('clf', MLPClassifier(verbose=True,alpha=1e-3))])
parameters_clf = {'clf__max_iter': (3,4,5),
                  'clf__alpha': (1e-4,1e-3),
                  
                  
                  
                  
                  
               
               
 }
gs_clf_svm = GridSearchCV(clf_pipeline, parameters_clf, n_jobs=-1,verbose=10)
gs_clf_svm = gs_clf_svm.fit(X_train, y_train)
print(gs_clf_svm.best_score_)
print(gs_clf_svm.best_params_)

## Gradient Boosting

In [7]:
from sklearn.ensemble import GradientBoostingClassifier
clf =GradientBoostingClassifier(loss='deviance', learning_rate=0.1, n_estimators=10,
                                subsample=1.0, criterion='friedman_mse', min_samples_split=2, min_samples_leaf=1,
                                min_weight_fraction_leaf=0.0, max_depth=16, min_impurity_decrease=0.0,
                                min_impurity_split=None, init=None, random_state=None, max_features=None,
                                verbose=10, max_leaf_nodes=None, warm_start=False, presort='auto', validation_fraction=0.1,
                                n_iter_no_change=None, tol=0.001)
clf = clf.fit(X_train, y_train)
predicted = clf.predict(X_valid)
np.mean(predicted == y_valid)

      Iter       Train Loss   Remaining Time 
         1      139594.0769            4.04m
         2      131935.3108            3.68m


KeyboardInterrupt: 

## LogReg

In [ ]:
clf = LogisticRegression(penalty='l2', dual=False, tol=0.001,
                   C=1.0, fit_intercept=True, intercept_scaling=1,
                   class_weight=None, random_state=None, solver='newton-cg',
                   max_iter=100, multi_class='auto', verbose=10,
                   warm_start=False, n_jobs=-1, l1_ratio=None)
clf = clf.fit(X_train, y_train)
predicted = clf.predict(X_valid)
np.mean(predicted == y_valid)

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=20, criterion='gini', max_depth=None, min_samples_split=3,
                             min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto',
                             max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, 
                             bootstrap=True, oob_score=False, n_jobs=-1, random_state=None, verbose=10,
                             warm_start=False, class_weight=None)
clf = clf.fit(X_train, y_train)
predicted = clf.predict(X_valid)
np.mean(predicted == y_valid)

## Kfold

In [29]:
def kfold(clf,data,labels,k):
        i=0
        score=0
        batch_size=np.floor(data[:,1].size/k)
        kf = KFold(n_splits=k)
        kf.get_n_splits(train_data)
        for train_index, test_index in kf.split(data):
            training_data= data[train_index]
            
            training_label=labels[train_index]
            
            test_data= data[test_index]
            test_labels = labels[test_index]
            clf= clf.fit(training_data,training_label)
            preds = clf.predict(test_data)
            print("\nSegmentation #",i+1," Results:")
            print(np.mean(preds == test_labels))
            score+=np.mean(preds == test_labels)
            i+=1
            
        
        print("\nAverage Accuracy Score: ",score/k,"%")

In [30]:
kfold(clf,train_data,train_label,10)


Segmentation # 1  Results:
0.572

Segmentation # 2  Results:
0.5718571428571428

Segmentation # 3  Results:
0.577

Segmentation # 4  Results:
0.5892857142857143

Segmentation # 5  Results:
0.5675714285714286

Segmentation # 6  Results:
0.5732857142857143

Segmentation # 7  Results:
0.5782857142857143

Segmentation # 8  Results:
0.5708571428571428

Segmentation # 9  Results:
0.5727142857142857

Segmentation # 10  Results:
0.5804285714285714

Average Accuracy Score:  0.5753285714285715 %


## TEST DATA PREDS

In [43]:
print('**Loading data**')
df = pd.read_csv('./reddit_test.csv')

print('**Preprocessing data**')
df['clean_comments'] = df['comments'].replace('http\S', '', regex=True).replace('www\S', '', regex=True)
df['clean_comments'] = df['clean_comments'].str.replace("[^0-9a-zA-Z#+_]", " ")
df['clean_comments'] = df['clean_comments'].str.replace("&(\w);", " ")
#df['clean_comments'] = df['clean_comments'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))
tokenized_comments = df['clean_comments'].apply(lambda x: x.split())

stemmer = PorterStemmer()
tokenized_comments = tokenized_comments.apply(lambda x: [stemmer.stem(word)for word in x])

lem = WordNetLemmatizer()
tokenized_comments = tokenized_comments.apply(lambda x: [lem.lemmatize(word) for word in x])
for i in range(len(tokenized_comments)):
    tokenized_comments[i] = ' '.join(tokenized_comments[i])

df['clean_comments'] = tokenized_comments



#tfidf_vectorizer = TfidfVectorizer(max_df=0.99, min_df=2, smooth_idf = True, norm='l2', ngram_range=(1, 1), stop_words='english')
# TF-IDF feature matrix
tfidf = tfidf_vectorizer.transform(df['clean_comments'])
test_data = tfidf


**Loading data**
**Preprocessing data**


In [44]:
print(test_data.shape)
clf = MLPClassifier(hidden_layer_sizes=(95,), activation='relu', solver='adam',
                    alpha=0.00005, batch_size=200, learning_rate='invscaling',
                    learning_rate_init=0.0009, power_t=0.9, max_iter=4, shuffle=True,
                    random_state=False, tol=0.1, verbose=True, warm_start=False,
                    momentum=0.9, nesterovs_momentum=True, early_stopping=False,
                    validation_fraction=0.8, beta_1=0.9, beta_2=0.99, epsilon=1e-8,
                    n_iter_no_change=1)
clf = clf.fit(train_data, train_label)


(30000, 24567)
Iteration 1, loss = 2.49909279
Iteration 2, loss = 1.56542284
Iteration 3, loss = 1.24019864
Iteration 4, loss = 1.06411132


C:\Users\alloc\Anaconda3\envs\comp551a1\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (4) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [45]:
res= clf.predict(test_data)
np.savetxt("predictions.csv", np.vstack((df['id'], LE.inverse_transform(res))).T, delimiter=",",fmt='%s')

In [ ]:
print(train_data.shape)